# Feature Creation Notebook

In this notebook, i'll document the feature creation process

We import the data into a pandas dataframe

In [23]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_0b8917382c9d414ca2a51a83a0262bbd = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='U5TzTCGqnkwBtVgDzW5AafWuy6nLCgYLuwRi9HmRLOUh',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_0b8917382c9d414ca2a51a83a0262bbd.get_object(Bucket='loadingdataandviewingdatapandasan-donotdelete-pr-78pi1hg6bcwa5p',Key='creditcard.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)

The main goal is to fit the model either with the dataframes that were undersample and oversample (in order for our models to detect the patterns) and test it on the original testing set.

In [24]:
import numpy as np # linear algebra
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

print('No Frauds', round(df_data_1['Class'].value_counts()[0]/len(df_data_1) * 100,2), '% of the dataset')
print('Frauds', round(df_data_1['Class'].value_counts()[1]/len(df_data_1) * 100,2), '% of the dataset')

X = df_data_1.drop('Class', axis=1)
y = df_data_1['Class']

sss = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

for train_index, test_index in sss.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    original_Xtrain, original_Xtest = X.iloc[train_index], X.iloc[test_index]
    original_ytrain, original_ytest = y.iloc[train_index], y.iloc[test_index]

    # Turn into an array
original_Xtrain = original_Xtrain.values
original_Xtest = original_Xtest.values
original_ytrain = original_ytrain.values
original_ytest = original_ytest.values

# See if both the train and test label distribution are similarly distributed
train_unique_label, train_counts_label = np.unique(original_ytrain, return_counts=True)
test_unique_label, test_counts_label = np.unique(original_ytest, return_counts=True)

No Frauds 99.83 % of the dataset
Frauds 0.17 % of the dataset
Train: [ 30473  30496  31002 ... 284804 284805 284806] Test: [    0     1     2 ... 57017 57018 57019]
Train: [     0      1      2 ... 284804 284805 284806] Test: [ 30473  30496  31002 ... 113964 113965 113966]
Train: [     0      1      2 ... 284804 284805 284806] Test: [ 81609  82400  83053 ... 170946 170947 170948]
Train: [     0      1      2 ... 284804 284805 284806] Test: [150654 150660 150661 ... 227866 227867 227868]
Train: [     0      1      2 ... 227866 227867 227868] Test: [212516 212644 213092 ... 284804 284805 284806]


In this phase of the project we will implment "Random Under Sampling" which basically 
consists of removing data in order to have a more balanced dataset and thus avoiding
our models to overfitting.


In [25]:
df_data_1 = df_data_1.sample(frac=1)

# amount of fraud classes 492 rows.
fraud_df = df_data_1.loc[df_data_1['Class'] == 1]
non_fraud_df = df_data_1.loc[df_data_1['Class'] == 0][:492]

normal_distributed_df = pd.concat([fraud_df, non_fraud_df])

# Shuffle dataframe rows
new_df = normal_distributed_df.sample(frac=1, random_state=42)

The total number of rows is now 984.

In [28]:
new_df.shape

(984, 31)